# Understanding Stress Through Sleep Patterns 
by Yuheng Ouyang, Eduardo Rivera, Harpreet Singh, Songyang Yu 2025/11/19

## 1. Summary

In this project, we explore how an individual’s sleep wellness relates to their reported stress level using the Sleep Health and Lifestyle Dataset. Our goal is to investigate whether sleep patterns can help explain or predict how stressed a person feels. To do this, we perform exploratory data analysis to understand trends in the data and examine whether shorter or irregular sleep is associated with higher stress. We then build a predictive model to evaluate how well sleep duration and related lifestyle variables can estimate a person’s stress level. This analysis aims to provide insight into the connection between sleep habits and stress, and to identify whether simple, easily measurable lifestyle information can meaningfully contribute to stress assessment.

## 2. Introduction

Stress is a major public health concern, there are surveys consistently report that more than 1/3 of adults experience high stress on a regular basis (Sleep Foundation, 2025). Chronic stress has been linked to anxiety, cardiovascular disease, weakened immunity, and reduced productivity. At the same time, insufficient sleep is extremely common—recent population studies suggest that about 30 % of adults sleep fewer than the recommended 7 hours per night (Centers for Disease Control and Prevention [CDC], 2024). Because sleep plays a crucial role in emotional regulation, cognitive performance, and physical recovery, disruptions in sleep duration are believed to contribute directly to increased stress levels.

In this project, we ask whether information about a person’s sleep duration and daily lifestyle habits can help explain or predict their self-reported stress level. This question is important because stress assessments typically rely on subjective questionnaires or clinical evaluations, which can vary among individuals and are difficult to scale. If simple, easy-to-measure variables—such as how long someone sleeps or how active they are during the day—can provide meaningful insight into stress, this could support early identification of at-risk individuals and inform recommendations for healthier sleep and lifestyle patterns.

To explore this question, we analyze the Sleep Health and Lifestyle Dataset (Varishabatool, n.d.), which contains information on individuals’ sleep duration, sleep quality, physical activity, demographic attributes, and self-reported stress. This dataset allows us to examine behavioral patterns related to sleep, identify lifestyle factors that correlate with stress, and evaluate whether a predictive model can estimate stress levels using accessible and non-clinical lifestyle data.

## 3. Methods & Results

### 3.1. Imports

In [1]:
import pandas as pd
import altair as alt

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

### 3.2. Data Loading and Preprocessing

The Sleep Health and Lifestyle Dataset is publicly released on [Kaggle](https://www.kaggle.com/datasets/varishabatool/disorder) under [CC0](https://creativecommons.org/publicdomain/zero/1.0/) license. We use one of its [GitHub mirror](https://raw.githubusercontent.com/Muhanad-husn/Sleep-Health-and-Lifestyle/main/data.csv) (Husn, 2023) in our analysis to avoid Kaggle authentication.

In [2]:
url = "https://raw.githubusercontent.com/Muhanad-husn/Sleep-Health-and-Lifestyle/main/data.csv"
df = pd.read_csv(url, index_col = "Person ID")
df.head()

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
Person ID,,,,,,,,,,,,
1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [3]:
df.shape

(374, 12)

The dataset has 374 records and 12 features. In our analysis, we will evaluate the relationships between stress level and three features about sleep wellness. 
* `Sleep Duration` (numerical/continuous): hours of sleep per night
* `Quality of Sleep` (numerical/ordinal): self-reported quality of sleep
* `Sleep Disorder` (categorical): diagnosed sleep disorder (None, Insomnia, or Sleep Apnea)

The target column `Stress Level` is the level of stress assessed through interviews or questionnaires. The values range from 3 (minimum) to 8 (maximum). Our model will predict the stress level as numeric scores through the input features. 

To prepare the data for analysis, we first drop the unused columns. 

In [4]:
df_clean = df.copy()
df_clean = df_clean[["Sleep Duration",
                     "Quality of Sleep",
                     "Sleep Disorder",
                     "Stress Level"]]
df_clean.head()

,Sleep Duration,Quality of Sleep,Sleep Disorder,Stress Level
Person ID,,,,
1,6.1,6,NaN,6
2,6.2,6,NaN,8
3,6.2,6,NaN,8
4,5.9,4,Sleep Apnea,8
5,5.9,4,Sleep Apnea,8


The `NaN` values in the `Sleep Disorder` column means the individual has not been diagnosed with a sleep disorder. We replace the `NaN` values with a string value `No Disorder`.

In [5]:
df_clean[["Sleep Disorder"]] = df_clean[["Sleep Disorder"]].fillna("No Disorder")
df_clean.head()

,Sleep Duration,Quality of Sleep,Sleep Disorder,Stress Level
Person ID,,,,
1,6.1,6,No Disorder,6
2,6.2,6,No Disorder,8
3,6.2,6,No Disorder,8
4,5.9,4,Sleep Apnea,8
5,5.9,4,Sleep Apnea,8


We retain local copies of the original and preprocessed data. 

In [6]:
df.to_csv('data/data.csv')
df_clean.to_csv('data/data_clean.csv')

### 3.3. Data Spltting

The data is splitinto train (80%) and test (20%) sets. The train set contains 299 records. 

In [7]:
train_df, test_df = train_test_split(df_clean, test_size=0.2)
train_df.shape

(299, 4)

The below shows a summary of the train data split.

In [8]:
df_clean.describe(include="all")

,Sleep Duration,Quality of Sleep,Sleep Disorder,Stress Level
count,374.000000,374.000000,374,374.000000
unique,NaN,NaN,3,NaN
top,NaN,NaN,No Disorder,NaN
freq,NaN,NaN,219,NaN
mean,7.132086,7.312834,NaN,5.385027
std,0.795657,1.196956,NaN,1.774526
min,5.800000,4.000000,NaN,3.000000
25%,6.400000,6.000000,NaN,4.000000
50%,7.200000,7.000000,NaN,5.000000
75%,7.800000,8.000000,NaN,7.000000


### 3.4. Exploratory Data Analysis (EDA)

In [9]:
## on train split only to follow golden rule
## four figures: each vs. stress level
## label the figures with numbers

### 3.5. Baseline Model

We separate the feature columns and the target column. 

In [10]:
X_train = train_df.drop(columns=["Stress Level"])
y_train = train_df["Stress Level"]
X_test = test_df.drop(columns=["Stress Level"])
y_test = test_df["Stress Level"]

We train a `DummyClassifier` as our baseline model. 

In [11]:
dummy = DummyClassifier()

dummy_scores = cross_validate(
    dummy,
    X_train,
    y_train,
    cv=5,
    return_train_score=True
)

dummy_df = pd.DataFrame(dummy_scores)
dummy_df

,fit_time,score_time,test_score,train_score
0,0.002016,0.001007,0.20000,0.200837
1,0.002719,0.001370,0.20000,0.200837
2,0.026988,0.007000,0.20000,0.200837
3,0.003000,0.000999,0.20000,0.200837
4,0.002001,0.000999,0.20339,0.200000


Our baseline model shows a validation accuracy of around 20%. We therefore aim for a test score higher than that. 

### 3.6. Column Transformation

We will perform standard scaling on `Sleep Duration` and one-hot encoding on `Sleep Disorder`. `Quality of Sleep` contains naturally ordered integers and therefore does not require transformation. 

We set up a `ColumnTransformer` to use in the pipeline.  

In [12]:
preprocesser = make_column_transformer(
    (StandardScaler(), ["Sleep Duration"]),
    (OneHotEncoder(), ["Sleep Disorder"]),
)

preprocesser

,transformers,"[('standardscaler', ...), ('onehotencoder', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


### 3.7. Model Selection

In [13]:
## use a model to predict stress level and find correlation 
## between stress and other features 

## i think either ridge or svm is fine

### 3.8. Training and Validation

In [14]:
## train the model and simply display the cv results

# pipe = make_pipeline(
#     preprocesser,
#     ## the model
# )

### 3.9. Model Evaluation

In [15]:
## test the model and report accuracy

## 4. Discussion

## 5. Refereneces

Centers for Disease Control and Prevention. (2024). *FastStats: Sleep in adults*. https://www.cdc.gov/sleep/data-research/facts-stats/adults-sleep-facts-and-stats.html

Husn, M. (2023). *Sleep health and lifestyle* [Data set]. GitHub. https://raw.githubusercontent.com/Muhanad-husn/Sleep-Health-and-Lifestyle/main/data.csv

Sleep Foundation. (2025). 100+ sleep statistics – Facts and data about sleep 2024. https://www.sleepfoundation.org/how-sleep-works/sleep-facts-statistics

Varishabatool. (n.d.). *Sleep health and lifestyle dataset* [Data set]. Kaggle. https://www.kaggle.com/datasets/varishabatool/disorder
